In [1]:
import pandas as pd
from pathlib import Path

# ----- 1. 경로 설정 -----------------------------------------------------------
csv_path  = Path("181_metabolite_clinical.csv")   # 업로드된 원본 CSV
set_path  = Path("metabolite_mapping.set")        # 업로드된 매핑 파일
out_path  = Path("metabolite_mapping_intersection.set")  # ★ 새로 만들 결과

# ----- 2. CSV의 대사체 열 이름 수집 ------------------------------------------
# 헤더만 읽으면 속도가 가장 빠름
metab_cols = set(pd.read_csv(csv_path, nrows=0).columns)

# ----- 3. .set 파일 필터링 ----------------------------------------------------
with set_path.open() as f:
    lines = [ln.rstrip("\n") for ln in f if ln.strip()]        # 공백·빈줄 제거

# 탭으로 split → 두 번째 토큰(대사체 약어)이 CSV에 있으면 keep
filtered = [ln for ln in lines if ln.split("\t")[1] in metab_cols]

# ----- 4. 결과 저장 -----------------------------------------------------------
out_path.write_text("\n".join(filtered) + "\n", encoding="utf-8")

print(f"{len(filtered):,} records written to {out_path.name}")

392 records written to metabolite_mapping_intersection.set


In [2]:
metab_cols

{'ADMA',
 'AFP',
 'Ac-Orn',
 'Ala',
 'Arg',
 'Asn',
 'Asp',
 'Azelaic acid',
 'C0',
 'C10',
 'C10:1',
 'C10:2',
 'C12',
 'C12-DC',
 'C12:1',
 'C14',
 'C14:1',
 'C14:1-OH',
 'C14:2',
 'C14:2-OH',
 'C16',
 'C16-OH',
 'C16:1',
 'C16:1-OH',
 'C16:2',
 'C16:2-OH',
 'C18',
 'C18:1',
 'C18:1-OH',
 'C18:2',
 'C2',
 'C3',
 'C3-DC (C4-OH)',
 'C3-OH',
 'C3:1',
 'C4',
 'C4:1',
 'C5',
 'C5-DC (C6-OH)',
 'C5-M-DC',
 'C5-OH (C3-DC-M)',
 'C5:1',
 'C5:1-DC',
 'C6 (C4:1-DC)',
 'C6:1',
 'C7-DC',
 'C8',
 'C9',
 'Cit',
 'Creatinine',
 'Gln',
 'Glu',
 'Gly',
 'Group',
 'H1',
 'HCC',
 'His',
 'Ile',
 'Kynurenine',
 'Leu',
 'Lys',
 'Met',
 'Met-SO',
 'Name',
 'Orn',
 'PC aa C24:0',
 'PC aa C26:0',
 'PC aa C28:1',
 'PC aa C30:0',
 'PC aa C30:2',
 'PC aa C32:0',
 'PC aa C32:1',
 'PC aa C32:2',
 'PC aa C32:3',
 'PC aa C34:1',
 'PC aa C34:2',
 'PC aa C34:3',
 'PC aa C34:4',
 'PC aa C36:0',
 'PC aa C36:1',
 'PC aa C36:2',
 'PC aa C36:3',
 'PC aa C36:4',
 'PC aa C36:5',
 'PC aa C36:6',
 'PC aa C38:0',
 'PC aa C38:1

In [3]:
filtered

['map00071\tC16',
 'map00120\tGly',
 'map00120\tTaurine',
 'map00130\tTyr',
 'map00220\tArg',
 'map00220\tAsp',
 'map00220\tCit',
 'map00220\tGln',
 'map00220\tGlu',
 'map00220\tOrn',
 'map00220\tAc-Orn',
 'map00230\tGln',
 'map00230\tGly',
 'map00240\tGln',
 'map00250\tAla',
 'map00250\tAsn',
 'map00250\tAsp',
 'map00250\tGln',
 'map00250\tGlu',
 'map00260\tAsp',
 'map00260\tGly',
 'map00260\tSer',
 'map00260\tThr',
 'map00260\tTrp',
 'map00261\tArg',
 'map00261\tAsp',
 'map00261\tSer',
 'map00261\tThr',
 'map00261\tTyr',
 'map00270\tAla',
 'map00270\tAsp',
 'map00270\tMet',
 'map00270\tSer',
 'map00270\tMet-SO',
 'map00280\tIle',
 'map00280\tLeu',
 'map00280\tVal',
 'map00290\tIle',
 'map00290\tLeu',
 'map00290\tThr',
 'map00290\tVal',
 'map00300\tAsp',
 'map00300\tLys',
 'map00300\talpha-AAA',
 'map00310\tGly',
 'map00310\tLys',
 'map00310\talpha-AAA',
 'map00311\tVal',
 'map00311\talpha-AAA',
 'map00330\tArg',
 'map00330\tGlu',
 'map00330\tOrn',
 'map00330\tPro',
 'map00330\tt4-OH-

In [2]:

import pandas as pd

def generate_layerinfo(mapping_path: str,
                       output_path: str = "layerinfo.csv",
                       default_layer_num: int = 1) -> pd.DataFrame:
    """
    metabolite_mapping.set (tab-separated) 파일을 읽어
    pathway 코드별로 metabolite 개수를 세고,
    모두 같은 default_layer_num 값을 부여한 뒤
    layerinfo.csv 로 저장합니다.

    Parameters
    ----------
    mapping_path : str
        pathway-to-metabolite 매핑 파일 경로 (예: "metabolite_mapping.set").
    output_path : str, optional
        생성할 CSV 파일명, by default "layerinfo.csv".
    default_layer_num : int, optional
        pathway마다 부여할 레이어 수, by default 1.

    Returns
    -------
    pd.DataFrame
        생성된 DataFrame (columns: pathway, node_num, layer_num).
    """
    # 1) 매핑 파일 읽기
    df = pd.read_csv(mapping_path,
                     sep='\t',
                     header=None,
                     names=["pathway", "metabolite"],
                     dtype={"pathway": str, "metabolite": str})

    # 2) pathway별 metabolite 개수 집계
    agg = (
        df
        .groupby("pathway")
        .size()
        .reset_index(name="node_num")
    )

    # 3) layer_num 컬럼 추가
    agg["layer_num"] = default_layer_num
    agg['node_num'] = 32

    # 4) CSV 저장
    agg.to_csv(output_path, index=False)

    return agg


filepath = 'metabolite_mapping_intersection.set'
layerinfo = generate_layerinfo(filepath, output_path='layerinfo.csv', default_layer_num=1)